In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import time
import os

In [7]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()
def count_pos(arr):
  count = 0
  for num in arr:
    if num > 0:
        count += 1

  print("Number of positive elements in the array:", count)


In [8]:
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    curvature = (dx_dt * d2y_dt2 - d2x_dt2 * dy_dt) / (dx_dt**2 + dy_dt**2)**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded
def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)

def get_integral_curvature(i,x,y, r):
    # Calculate the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]

    return int_curv

    return int_curv


def get_area_measure(contour, i, radius, thresh, prev_center, prev_mask):
    # Compute current circle center and mask
    center = (contour[i][0][0], contour[i][0][1])
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, center, radius, (255, 255, 255), -1)
    
    # Compute mask difference to find changed pixels
    diff_mask = cv2.absdiff(circle_mask, prev_mask)
    changed_pixels = np.where(diff_mask > 0)
    
    # Compute intersection area using only changed pixels
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    changed_intersection_mask = intersection_mask[changed_pixels]
    intersection_area = np.sum(changed_intersection_mask) / 255
    
    # Update previous center and mask
    prev_center = center
    prev_mask = circle_mask
    
    # Compute area fraction
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area
    
    # Return area fraction
    return intersection_fraction


def get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters):
    curvature_arr = []
    # Approximate the contour with a circle centered at the current point
    (cx, cy) = x[i], y[i]
    # Calculate the length of the part of the circle's circumference that is inside the object
    chord_length = 2 * np.sqrt(radius**2 - ((radius**2-(x[i]-cx)**2-(y[i]-cy)**2))/4)
    arc_length = 2 * np.arcsin(chord_length / (2 * radius))
    circle_perimeters.append(arc_length)
    if len(circle_perimeters) > 0:
    # Calculate the average perimeter of the fitted circles
      circle_perimeter = np.mean(circle_perimeters)
    else:
                    # Use the radius as an estimate for the perimeter
      circle_perimeter = 2 * np.pi * radius

                # Calculate perimeter of the contour
    contour_perimeter = cv2.arcLength(contour, True)
                # Calculate the fraction of the circle's perimeter contained inside the object
    fraction = circle_perimeter / contour_perimeter
    arc_len = fraction * arc_length


        
    return arc_len

def get_curvatures(contours, radius_arr,thresh):
    curvature_arr=[]
    
    # Loop over each contour
    for contour in contours:
        curr = []

        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the maximum and minimum x and y coordinates of the contour
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        # Compute the thresholded image for the current contour
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        for radius in radius_arr:
          prev_center = contour[0][0]
          prev_mask = np.zeros_like(gray)
          circle_perimeters = []
          arc_length = 0  # Initialize arc_length to 0
          curr = []
          for i in range(len(contour)):


            curr_area = get_area_measure(contour,i,radius,thresh,prev_center,prev_mask)

            curr.append(curr_area)

          curvature_arr.append(curr)
    return curvature_arr

def to_hist(currvature_arr):
  currvature_arr = np.array(currvature_arr)
  x = np.arange(currvature_arr.shape[1]) # x-axis is the number of contour points

  y = np.arange(currvature_arr.shape[0]) # y-axis is the different scales

  X, Y = np.meshgrid(x, y)
  hist_values = []
  
  for i in range(currvature_arr.shape[0]):
      plt.figure()
      counts, bin_edges = np.histogram(currvature_arr[i], bins=20) # adjust the number of bins as needed
      hist_values.append(counts)
  return hist_values

In [14]:
# specify path to the folder containing image files
radius_arr = np.arange(10, 131, 5)
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"

counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue

    if counter == 1:
        break
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        # try:
            
        start_time = time.time()
        # Load the image
        img = cv2.imread(filepath)
        # set the new size
        new_size = (250, 250)
        # resize the image
        # resize the image
        img = cv2.resize(img, new_size)

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Threshold the image to get a binary image
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Find the contours in the image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        radius_arr = np.arange(10, 131, 5)
        merged_contours = []
        for contour in contours:
            merged_contours += contour.tolist()
        oc = np.array([merged_contours])

        currvature_arr = get_curvatures(oc,radius_arr, thresh)
        curvature_arr = np.array(currvature_arr)
        # print(currvature_arr)
        
        x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
        y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales

        X, Y = np.meshgrid(x, y)
        hist_values = []
        for i in range(curvature_arr.shape[0]):
            counts, bin_edges = np.histogram(curvature_arr, bins=21) # adjust the number of bins as needed
            hist_values.append(counts)

        data = np.array([np.array(hist) for hist in hist_values])
        X2 = data.flatten().tolist()

        print(X2)
        features.append(X2)
        labels.append(foldername)
        end_time = time.time()
        total_time = end_time - start_time

        # except:
        #     pass

        counter += 1
        
    print(f"Finished plant: {foldername}")
    
# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("lf_data.csv", index=False)

[3, 2, 2, 2, 3, 0, 3, 8, 5, 6, 11, 14, 15, 7, 4, 4, 1, 2, 1, 1, 4, 4, 2, 3, 1, 4, 2, 2, 11, 6, 11, 8, 16, 7, 8, 2, 2, 1, 0, 3, 1, 4, 4, 3, 3, 2, 3, 2, 9, 7, 7, 11, 16, 9, 6, 4, 2, 0, 1, 1, 2, 3, 3, 6, 1, 5, 2, 1, 7, 5, 11, 11, 11, 6, 11, 7, 2, 1, 2, 2, 1, 2, 2, 2, 4, 2, 5, 3, 2, 5, 10, 10, 8, 7, 5, 8, 7, 5, 3, 4, 2, 2, 2, 2, 2, 4, 2, 6, 2, 2, 7, 8, 11, 8, 4, 3, 3, 6, 9, 5, 5, 2, 3, 2, 4, 2, 4, 4, 4, 2, 2, 6, 7, 11, 8, 5, 1, 3, 5, 5, 9, 5, 4, 5, 3, 3, 2, 4, 2, 5, 3, 2, 2, 5, 6, 5, 8, 8, 3, 1, 4, 5, 7, 4, 9, 10, 2, 3, 4, 0, 3, 4, 2, 3, 2, 3, 3, 5, 7, 8, 5, 4, 4, 7, 6, 4, 10, 9, 5, 1, 3, 0, 2, 3, 2, 4, 2, 2, 2, 3, 4, 6, 8, 6, 8, 10, 6, 9, 6, 11, 1, 3, 0, 0, 4, 0, 0, 2, 0, 3, 1, 4, 8, 2, 3, 3, 7, 10, 11, 13, 23, 1, 1, 4, 0, 0, 0, 2, 0, 0, 1, 3, 2, 2, 2, 4, 3, 2, 4, 6, 11, 50, 3, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 3, 2, 2, 0, 3, 2, 3, 3, 5, 66, 2, 1, 0, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 2, 1, 1, 4, 0, 3, 2, 77, 2, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 1, 87, 3, 0, 0, 0, 0, 0, 0, 0, 3

KeyboardInterrupt: 

In [14]:
# Load the image
img = cv2.imread('13002220995650.png')
# set the new size
new_size = (250, 250)

# resize the image
img = cv2.resize(img, new_size)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Threshold the image to get a binary image
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find the contours in the image
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

radius_arr = np.arange(10, 131, 5)
merged_contours = []
for contour in contours:
    merged_contours += contour.tolist()
oc = np.array([merged_contours])



In [15]:


start_time = time.time()

currvature_arr = get_curvatures(oc,radius_arr, thresh)
print(currvature_arr)

end_time = time.time()

total_time = end_time - start_time

print(f"Total run time: {total_time} seconds")

[[0.4217793233044746, 0.475892003955719, 0.5325761291777031, 0.47233442287484134, 0.4246253881691768, 0.46611801803878145, 0.44781832026837215, 0.48354392396476464, 0.6045016807146051, 0.6434977624572782, 0.5738690351971533, 0.49471597683278395, 0.4757546938087378, 0.5940036958408574, 0.6349845333444412, 0.616347801576896, 0.540253014668018, 0.5305539251948884, 0.45415955251077866, 0.437445162800971, 0.4971251457752731, 0.5385054309791659, 0.46021368171858806, 0.5047770657843188, 0.48471730158442256, 0.544235008930474, 0.5720715205457625, 0.5402405319273834, 0.5720715205457625, 0.4572677549288086, 0.4772401399442622, 0.4390429536022073, 0.544235008930474, 0.5665167009633394, 0.5028547237265812, 0.5192445621798879, 0.4406157789221743, 0.5906957695726729, 0.5948400394633795, 0.4691138757910995, 0.4741818684887708, 0.5187452525545015, 0.45433431087966386, 0.5307286835637736, 0.619855451695235, 0.6580526380372899, 0.6389540448662624, 0.5275455847019357, 0.4766160029125292, 0.50543865103795